In [1]:
import wrds
import pandas as pd

In [4]:
db = wrds.Connection()

Enter your WRDS username [sunmin]: sunmin
Enter your password: ········


There was an error with your password.


OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: FATAL:  PAM authentication failed for user "sunmin"

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# Define the start and end dates for the data
years = ['2011','2012','2013','2014','2015','2016', '2017', '2018', '2019', '2020', '2021']
start_dates = '-01-01'
end_dates = '-12-31'
start_time = '09:30'
end_time = '16:00'


In [4]:
# Define the ticker symbol for the stock
tickers = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL', 'GOOG', 'BRK.B', 'UNH', 'JNJ', 'XOM', 'JPM', 'META','V', 'PG', 'NVDA', 'HD', 'CVX', 'LLY', 'MA', 'ABBV']

In [5]:
for year in years :
    start_date = year + start_dates
    end_date = year + end_dates
    root_data = 'taqm_' + year +'.ctm_' +year
    print(root_data)
    for ticker in tickers :
        # Construct the SQL query
        query = f"""
        SELECT TO_CHAR(date_trunc('day', date), 'YYYY-MM-DD') AS day,
               TO_CHAR(date_trunc('minute', time_m - (EXTRACT(minute FROM time_m)::int %% 5) * interval '1 minute'), 'HH24:MI:SS') AS one_min,
               min(price) AS min_price,
               max(price) AS max_price,
               CASE WHEN sum(size) != 0 THEN
                   sum(price * size) / sum(size)
               ELSE 
                   0
               END AS weighted_avg,  
               avg(price) AS avg_price,
               sum(size) AS volume,
               (array_agg(price ORDER BY time_m))[1] as first_price,
               (array_agg(price ORDER BY time_m desc))[1] as last_price
        FROM {root_data}
        WHERE sym_root='{ticker}' AND date >= '{start_date}' AND date <= '{end_date}' AND time_m >= '{start_time}' AND time_m <= '{end_time}' 
              AND tr_scond !~'[ZQTUMO6]' AND tr_corr='00' AND sym_suffix IS NULL
        GROUP BY date_trunc('day', date), date_trunc('minute', time_m - (EXTRACT(minute FROM time_m)::int %% 5) * interval '1 minute')
        ORDER BY day asc, one_min asc
        """
    
        # Retrieve the data from the WRDS database
        data = db.raw_sql(query)
        
        # Construct the file name and path
        file_name = f"{year}_{ticker}.csv"
        folder_path = "stock_data"

        # Save the DataFrame to a CSV file in the specified folder
        data.to_csv(folder_path + "/" + file_name, index=False)

taqm_2011.ctm_2011
taqm_2012.ctm_2012
taqm_2013.ctm_2013
taqm_2014.ctm_2014
taqm_2015.ctm_2015
taqm_2016.ctm_2016
taqm_2017.ctm_2017
taqm_2018.ctm_2018
taqm_2019.ctm_2019
taqm_2020.ctm_2020
taqm_2021.ctm_2021


In [6]:
# Define the start and end dates for the data
years = ['2011']
start_dates = '-01-01'
end_dates = '-12-31'
start_time = '09:30'
end_time = '16:00'# Define the ticker symbol for the stock
tickers = ['TSLA']

In [12]:
for year in years :
    start_date = year + start_dates
    end_date = year + end_dates
    root_data = 'taqm_' + year +'.ctm_' +year
    print(root_data)
    for ticker in tickers :
        # Construct the SQL query
        query = f"""
        SELECT TO_CHAR(date_trunc('day', date), 'YYYY-MM-DD') AS day,
               TO_CHAR(date_trunc('minute', time_m - (EXTRACT(minute FROM time_m)::int %% 5) * interval '1 minute'), 'HH24:MI:SS') AS five_min,
               min(price) AS min_price,
               max(price) AS max_price,
               CASE WHEN sum(size) != 0 THEN
                   sum(price * size) / sum(size)
               ELSE 
                   0
               END AS weighted_avg,  
               avg(price) AS avg_price,
               sum(size) AS volume,
               (array_agg(price ORDER BY time_m))[1] as first_price,
               (array_agg(price ORDER BY time_m desc))[1] as last_price
        FROM {root_data}
        WHERE sym_root='{ticker}' AND date >= '{start_date}' AND date <= '{end_date}' AND time_m >= '{start_time}' AND time_m <= '{end_time}' 
              AND tr_scond !~'[ZQTUMO6]' AND tr_corr='00'
        GROUP BY date_trunc('day', date), date_trunc('minute', time_m - (EXTRACT(minute FROM time_m)::int %% 5) * interval '1 minute')
        ORDER BY day asc, five_min asc
        """
    
        # Retrieve the data from the WRDS database
        data = db.raw_sql(query)
        
        # Construct the file name and path
        file_name = f"{year}_{ticker}.csv"
        folder_path = "stock_data"

        # Save the DataFrame to a CSV file in the specified folder
        data.to_csv(folder_path + "/" + file_name, index=False)

taqm_2011.ctm_2011
